In [33]:
import re
import pandas as pd
import os
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."

def split_before_and_after_hyphen(file_name):
    parts = file_name.split('-', 1)
    if len(parts) == 2:
        before_hyphen, after_hyphen = parts
    else:
        before_hyphen, after_hyphen = file_name, ""
    return before_hyphen.strip(), after_hyphen.strip()

# Function to parse the test cases from the text file


def parse_test_cases(file_name):
    # with open(file_name, 'r') as file:
    #     content = file.read()
    content = read_file_content(file_name)
    base_name = os.path.basename(file_name)

    base_nam = os.path.splitext(base_name)[0]

    # Extract use case and test scenario names from the file name
    use_case_name, test_scenario_name = split_before_and_after_hyphen(base_nam)
    
    # Regular expression to extract test cases
    test_case_pattern = re.compile(
        r"Test Case: (.*?)\ntestCaseName: (.*?)\nobjective: (.*?)\ntestSteps:\n(.*?)\nexpectedResult: (.*?)\n",
        re.DOTALL
    )
    
    matches = test_case_pattern.findall(content)
    
    test_cases = []
    for match in matches:
        test_case_name = match[1].strip()
        objective = match[2].strip()
        test_steps = match[3].strip().replace('\n', ' ')
        expected_result = match[4].strip()
        
        test_cases.append({
            'Use Case Name': use_case_name,
            'Test Scenario Name': test_scenario_name,
            'Test Case Name': test_case_name,
            'Objective': objective,
            'Test Steps': test_steps,
            'Expected Result': expected_result
        })
    
    return test_cases

# Function to write the parsed test cases to an Excel file
def write_to_excel(test_cases, output_file):
    df = pd.DataFrame(test_cases)
    df.to_excel(output_file, index=False)


def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

# Function to sort filenames naturally by the extracted number
def natural_sort_key(filename):
    # Split the filename into parts: prefix and number
    parts = re.split(r'(\d+)', filename)
    # Convert numerical parts to integers for proper numeric sorting
    parts[1::2] = map(int, parts[1::2])
    return parts

# Sort filenames using the natural sort key

def parse_use_cases(file_name):
    base_name = os.path.basename(file_name)
    usecase_name = os.path.splitext(base_name)[0]
    usecase_specification  = read_file_content(file_name)
    result = {
                'usecase_name': usecase_name,
                'usecase_specification': usecase_specification
            }
    return result

In [34]:
import openpyxl

def create_new_sheet(file_path, sheet_name):
    # Load the existing workbook
    workbook = openpyxl.load_workbook(file_path)

    # Create a new sheet with the specified name
    workbook.create_sheet(title=sheet_name)

    # Save the workbook
    workbook.save(file_path)

def add_data_to_sheet(file_path, sheet_name, data):
    # Load the existing workbook
    workbook = openpyxl.load_workbook(file_path)

    # Check if the sheet exists, if not create it
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(title=sheet_name)

    # Add data to the sheet
    for row in data:
        sheet.append(row)

    # Save the workbook
    workbook.save(file_path)

def write_to_excel(test_cases, output_file, sheet_name='Sheet1'):
    # Convert the list of JSON objects to a DataFrame
    df = pd.DataFrame(test_cases)

    # Try to open the existing workbook
    try:
        with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
            # Check if the sheet already exists
            writer.book = openpyxl.load_workbook(output_file)
            if sheet_name in writer.book.sheetnames:
                # If the sheet exists, remove it
                writer.book.remove(writer.book[sheet_name])
            # Write the DataFrame to the specified sheet
            df.to_excel(writer, index=False, sheet_name=sheet_name)
    except FileNotFoundError:
        # If the file does not exist, create a new workbook and add the sheet
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name=sheet_name)

In [35]:
output_file = 'test_cases.xlsx'

testcase_folder_path = r'D:\Dissertation-GPT\evaluate\ResultSet\6.24\shopee - best\tc'
usecase_folder_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Shopee(new)"
testcase_file_list = os.listdir(testcase_folder_path)
testcase_sorted_files = sorted(testcase_file_list, key=natural_sort_key)
usecase_file_list = os.listdir(usecase_folder_path)

In [36]:
    
all_test_cases = []
all_use_case = []
for filename in testcase_sorted_files:
    file_path = os.path.join(testcase_folder_path, filename)
    print(file_path)
    if os.path.isfile(file_path):
        test_cases = parse_test_cases(file_path)
        print(test_cases)
        test_cases = parse_test_cases(file_path)
        all_test_cases.extend(test_cases)
for filename in usecase_file_list:
    file_path = os.path.join(usecase_folder_path, filename)
    print(file_path)
    all_use_case.append(parse_use_cases(file_path))

# create_new_sheet(output_file, "Use case Description")
# create_new_sheet(output_file,"Test scenarios and Corresponding Test Case")
# create_new_sheet(output_file,"Test Case Detail")

# headings = ['Use case name', "Use case specification"]
# add_headings_to_sheet(output_file, "Use case Description",headings)
# headings = ['Use case name', "Use case specification"]
# add_headings_to_sheet(output_file,"Test scenarios and Corresponding Test Case",headings)
# add_headings_to_sheet(output_file,"Test Case Detail",headings)

write_to_excel(all_use_case, output_file,"Use case Description")
write_to_excel(all_test_cases,output_file,"Test Case Detail")
write_to_excel(all_test_cases, output_file,"Test scenarios and Corresponding Test Case")

D:\Dissertation-GPT\evaluate\ResultSet\6.24\shopee - best\tc\add to cart-1. Successful Product Addition to Cart.txt
[{'Use Case Name': 'add to cart', 'Test Scenario Name': '1. Successful Product Addition to Cart', 'Test Case Name': 'Successful Product Addition to Cart Without Options', 'Objective': 'Verify that a user can successfully add a product without options to their shopping cart', 'Test Steps': "- Step 1: Log into the Shopee account.   - Step 2: Navigate to a product detail page for a product without options.   - Step 3: Click the 'Add to Cart' button.   - Step 4: Observe the confirmation message that the item has been added to the cart.   - Step 5: Check the cart icon to ensure it reflects the updated number of items.", 'Expected Result': 'The product is added to the shopping cart successfully, a confirmation message is displayed, and the cart icon shows the correct number of items.'}, {'Use Case Name': 'add to cart', 'Test Scenario Name': '1. Successful Product Addition to Ca

e:\ProgramData\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\phamn\AppData\Local\Temp\ipykernel_18820\122952651.py:38: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = openpyxl.load_workbook(output_file)
e:\ProgramData\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\phamn\AppData\Local\Temp\ipykernel_18820\122952651.py:38: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected